# *Resultados*
* **Xgboost**
* * nan -> median
* categoric(onehotcode)+Number
* R²: 0.9106 
* ⏱️ Tempo de execução: 4.43 segundos

In [1]:
# =====================================================
# 🏠 House Prices - Pipeline Aprimorado
# =====================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, 
                       message='Found unknown categories in columns')

pd.set_option('display.max_columns', None)
import time

In [2]:
# =====================================================
# 📁 1. Leitura dos dados
# =====================================================
dfo = pd.read_csv("/home/akel/PycharmProjects/Kaggle/HousePrices/data/train.csv")

In [19]:
df_train=dfo.copy()
# =====================================================
# 🧹 2. Pré-processamento inicial
# =====================================================
# remoção de colunas com muitos nulos (> 10%)
colnull_train=df_train.columns[(df_train.isnull().sum()/df_train.shape[0]>0.1)] # 
df_train=df_train.drop(columns=colnull_train,axis=1)

id_train=df_train['Id']

# obtendo nome das variáveis categóricas e numéricas
num_features = df_train.select_dtypes(include=['number']).columns.drop(['Id', 'SalePrice'])
cat_features = df_train.select_dtypes(include=['object']).columns

# =====================================================
# 🧩 3. Pré-processadores
# =====================================================
# NAN -> median
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# categoric -> binario onehotcode 
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first',
                             sparse_output=False,
                             handle_unknown='ignore'))
])

preprocessador = ColumnTransformer(transformers=[
    ('cat', cat_transformer, cat_features),
    ('num', num_transformer, num_features)   
],verbose_feature_names_out=False) 

model_XGB2 = Pipeline([ ('preprocess', preprocessador), 
                        ('model', XGBRegressor( objective='reg:squarederror', 
                                              n_estimators=1000, 
                                              learning_rate=0.1, 
                                              max_depth=6, 
                                              subsample=0.8, 
                                              colsample_bytree=0.8,
                                              random_state=42, n_jobs=-1 ))
                      ])

#função validacao_cruzada_parapipeline
def valida(Xo,yo,model,N=5):
    print("=" * 44)
    print(f"validação cruzada (K-Fold Cross Validation)")
    print("=" * 44)

    kf = KFold(n_splits=N, shuffle=True, random_state=42)
    r2_scores = []
    for i, (train_idx, test_idx) in enumerate(kf.split(Xo, yo), 1):
        X_train, X_val = Xo.iloc[train_idx], Xo.iloc[test_idx]
        y_train, y_val = yo.iloc[train_idx], yo.iloc[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        r2 = r2_score(y_val, y_pred)
        r2_scores.append(r2)
        print(f"Fold {i}: R² = {r2:.4f}")

    print(f"\n📊 R² médio: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
    return r2_scores

In [4]:
X=df_train.drop(['Id', 'SalePrice'], axis=1)
y=df_train['SalePrice']
start_time = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model_XGB2.fit(X_train,y_train)

y_xg2 = model_XGB2.predict(X_test)
end_time = time.time()

mae = mean_absolute_error(y_test, y_xg2)
mse = mean_squared_error(y_test, y_xg2)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_xg2)
resultados={
            'Modelo': '🤖 XGBOOST2_opt',
            'MAE': round(mae, 4),
            'RMSE': round(rmse, 4),
            'R²': round(r2, 4)
        }

for k, v in resultados.items():
    print(f"{k}: {v}")
print(f"⏱️ Tempo de execução: {end_time - start_time:.2f} segundos")


Modelo: 🤖 XGBOOST2_opt
MAE: 15887.7988
RMSE: 24974.7546
R²: 0.9106
⏱️ Tempo de execução: 5.37 segundos


In [20]:
#Validação_cruzada
r2=valida(X,y,model=model_XGB2,N=7)

validação cruzada (K-Fold Cross Validation)
Fold 1: R² = 0.9221
Fold 2: R² = 0.9087
Fold 3: R² = 0.9212
Fold 4: R² = 0.5953
Fold 5: R² = 0.8938
Fold 6: R² = 0.8864
Fold 7: R² = 0.9171

📊 R² médio: 0.8635 ± 0.1102
